### GridBatch and JaggedTensor
There are two fundamental classes in fVDB you will encounter frequently: `GridBatch` and `JaggedTensor`.

```python
fvdb.GridBatch
fvdb.JaggedTensor
```

#### GridBatch

A `GridBatch` is an indexing structure which maps the 3D $ijk$ coordinates of a set of **sparse** grids to integer offsets which can be used to look up elements in a data tensor that correspond with each voxel.

This mapping only exists for $ijk$ coordinates which are **active** in the space of a **sparse**, 3D grid.

&#x1F4A1; We call these 3D grids **sparse** because of this arbitrary nature to the topology of **active** voxels in the grid compared to **dense** grids where all voxels exist inside some regular extents in each dimension.

The figure below illustrates this $ijk$ mapping process for a `GridBatch` containing only a single grid.


<center>

<img src="img/gridbatch.svg"  alt="Image Index Grid" width="800"/>

</center>

In practice, `GridBatch` is an ordered collection of 1 or more of these 3D grids.  

&#x1F529;	At the level of technical implementation, these 3D grids are [**NanoVDB**](https://academysoftwarefoundation.github.io/openvdb/NanoVDB_MainPage.html) grids of the special `IndexGrid` type which only stores a unique **index** integer value at each active voxel location.  This **index** is an offset into some external data array, a tensor, (one that is not contained in the `IndexGrid`/`GridBatch` classes) where contiguous array members correspond to spatially nearby voxels.  `IndexGrid` will allow us to reference into this 'sidecar' tensor of data given a spatial $ijk$ set of coordinates.

Each grid member in a `GridBatch` can have different topologies, different numbers of active voxels and different voxel dimensions and origins per-grid.

<center>

<img src="img/gridbatch_concept.svg"  alt="Image Index Grid" width="900"/>

</center>

##### Images as 2D GridBatch


To help explain these concepts, let's consider how we might treat image data with this framework (an image can be thought of as a dense 2D grid after all).  

If an image is a 2D grid of pixels, we could imagine the position of each pixel could be expressed as $i,j$ coordinates.  For an RGB image, each pixel would contain 3 associated values $(R,G,B)$.

In this way, we could decompose an image into:

1.  an `IndexGrid` of $i,j$ coordinates
2.  a tensor of size $[NumberPixels, 3]$ (the flattened RGB elements for our data tensor)

The $i,j$ coordinates can be used with the `IndexGrid` to index into the list of RGB values to retrieve the RGB element for each pixel.  In our nomenclature, we'd say our data has element size 3 (RGB) and the number of elements in our data tensor is equal to the number of pixels in the image (in the illustration below the number of elements is 16).

<center>

<img src="img/image_index_grid_diagram.svg"  alt="Image Index Grid" width="800"/>

</center>

If we had a batch of *many* images, each of different sizes, we can imagine constructing a `GridBatch` as an ordered set of their `IndexGrid`s.

All the RGB elements would go into a sidecar data tensor where each array of data would have to be of a different length corresponding to the image size.  We call this list of different-length element data a **jagged tensor**.

<center>

<img src="img/grid_batch_diagram.svg"  alt="Grid Batch" width="600"/>

</center>

This is the essence of the relationship between `GridBatch` and `JaggedTensor` in fVDB.  But more on `JaggedTensor` later…

Lastly, it is important to know that each grid in the `GridBatch` will be on the same device and processed together by operators in mini-batch-like fashion.  

Let's put together our first `GridBatch` to see how it works.

In [1]:
# import the usual suspects and fvdb
import numpy as np
import torch
import fvdb

We will make a `GridBatch` of 8 grids, each with a different number of active voxels and all of those voxels' positions will be chosen randomly.  Further, each grid will have a randomly chosen origin in the 3D world space and a randomly chosen voxel size.

In [2]:
batch_size = 8
# Randomly generate different numbers of voxels we desire in each grid in our batch
num_voxels_per_grid = [np.random.randint(100, 1_000) for _ in range(batch_size)]

# A list of randomly generated 3D indices for each grid in our batch in the range [-512, 512]
ijks = [torch.randint(-512, 512, (num_voxels_per_grid[i], 3), device="cuda") for i in range(batch_size)]

# Create an fvdb.GridBatch from the list of indices!!
grid_batch = fvdb.gridbatch_from_ijk(
    fvdb.JaggedTensor(ijks),  # We'll explain JaggedTensor in a moment…
    # Random, different voxel sizes for each grid in our batch
    voxel_sizes=[np.random.rand(3).tolist() for _ in range(batch_size)],
    # Random, different grid origins for each grid in our batch
    origins=[np.random.rand(3).tolist() for _ in range(batch_size)],
)

There are many more convenient ways that fVDB provides to create a `GridBatch` besides building from lists of coordinate indexes such as building a `GridBatch` from **worldspace pointclouds, meshes or dense tensors**.

&#x1F4A1; The fVDB documentation has more useful examples for these cases using functions like `gridbatch_from_points`, `gridbatch_from_dense` and `gridbatch_from_mesh`.

In [3]:
# This grid will be on the GPU because the `ijks` were on that device
assert(grid_batch.device == ijks[0].device == torch.device('cuda:0'))

Each member of the batch has different voxel size dimensions, a different origin in space and different number of voxels

In [4]:
for i in range(grid_batch.grid_count):
    print(f"""Grid {i} has {grid_batch.num_voxels_at(i)} voxels,
        voxel size of {grid_batch.voxel_size_at(i).tolist()}
        and an origin of {grid_batch.origin_at(i).tolist()}""")

Grid 0 has 154 voxels,
        voxel size of [0.8820853233337402, 0.7485278248786926, 0.13592280447483063]
        and an origin of [0.04669933021068573, 0.6575663685798645, 0.4350457191467285]
Grid 1 has 485 voxels,
        voxel size of [0.8010919690132141, 0.8362209796905518, 0.13165147602558136]
        and an origin of [0.7419846057891846, 0.6002568006515503, 0.059590213000774384]
Grid 2 has 345 voxels,
        voxel size of [0.0776681900024414, 0.9747209548950195, 0.5741835832595825]
        and an origin of [0.042450230568647385, 0.9626051783561707, 0.9474310278892517]
Grid 3 has 978 voxels,
        voxel size of [0.40188148617744446, 0.7703567147254944, 0.3520265519618988]
        and an origin of [0.8566922545433044, 0.6265580058097839, 0.4814351797103882]
Grid 4 has 537 voxels,
        voxel size of [0.5046014189720154, 0.29932141304016113, 0.6225708723068237]
        and an origin of [0.8637191653251648, 0.055337607860565186, 0.8349815011024475]
Grid 5 has 291 voxels,
      

Let's examine some of the ways we can retrieve indices from a `GridBatch` based on $ijk$ coordinates.

In [5]:
# Let's retrieve a random ijk coordinate from each of the lists we used to make the grids
ijk_queries = fvdb.JaggedTensor([ijks[n][np.random.randint(len(ijks[n]))][None,:] for n  in range(grid_batch.grid_count)])

# Use the GridBatch to get indices into the sidecar feature array from the `ijk` coordinate in each grid
feature_indices = grid_batch.ijk_to_index(ijk_queries)
world_positions = grid_batch.grid_to_world(ijk_queries.float())
for i, (ijk, world_p, i_f) in enumerate(zip(ijk_queries, world_positions, feature_indices)):
    print(f"Grid {i}, feature Index at ijk {ijk.jdata.tolist()}, world-space {world_p.jdata.tolist()} : {i_f.jdata.item()}")

# NOTE: This GridBatch (Batch of IndexGrids) just expresses the topology of the grids and can be used to reference a sidecar flat array of features but we won't create this sidecar in this example…
# We can get the index into this hypothetical sidecar feature array with any `ijk` coordinate (if we ask for an `ijk` not in the grid, -1 is given as the index)

Grid 0, feature Index at ijk [[358, -169, -113]], world-space [[315.8332214355469, -125.8436279296875, -14.924229621887207]] : 97
Grid 1, feature Index at ijk [[-388, -12, 461]], world-space [[-310.0816955566406, -9.434394836425781, 60.750919342041016]] : 61
Grid 2, feature Index at ijk [[-258, -286, 306]], world-space [[-19.995941162109375, -277.80755615234375, 176.64761352539062]] : 63
Grid 3, feature Index at ijk [[-29, -44, 376]], world-space [[-10.797870635986328, -33.269134521484375, 132.84341430664062]] : 213
Grid 4, feature Index at ijk [[-71, -279, -303]], world-space [[-34.962982177734375, -83.45533752441406, -187.8040008544922]] : 49
Grid 5, feature Index at ijk [[505, 422, 218]], world-space [[215.74253845214844, 152.41024780273438, 43.96575164794922]] : 288
Grid 6, feature Index at ijk [[-223, 77, 408]], world-space [[-3.5291380882263184, 41.81416702270508, 88.72368621826172]] : 472
Grid 7, feature Index at ijk [[-385, -375, 497]], world-space [[-168.81019592285156, -143.5

#### JaggedTensor

`JaggedTensor` is the supporting element data (i.e. the 'sidecar' of data) that is paired with a `GridBatch`.  

You can think of `JaggedTensor` as consisting of an ordered list of PyTorch Tensors, one for each grid in the `GridBatch`.  Same as `GridBatch`, the Tensors are all on the same device and processed together in a mini-batch. 

In [6]:
# Here we create a list of Tensors, one for each grid in the batch with its number of active voxels and 8 random features per-voxel
list_of_features = [torch.randn(int(grid_batch.num_voxels[i]), 5, device=grid_batch.device) for i in range(grid_batch.grid_count)]
# Now we make a JaggedTensor out of this list of heterogeneous Tensors
features  = fvdb.JaggedTensor(list_of_features)

In the `JaggedTensor` above, you can see how we constructed it with a list of heterogeneously sized Tensors whose shapes were of the form:

$$[ [B1, E], [B2, E], [B3, E], …] ]$$

where the value of $Bn$ would be the number of active voxels in each grid of the `GridBatch` and $E$ is the number of elements (5 was chosen in this case).

&#x1F4A1; Note how each Tensor element in our `JaggedTensor` can have different numbers of active voxels (similar to `GridBatch`) but the same number of per-voxel elements.  This is distinctly different from the classic representation of 3D data in a PyTorch `Tensor` which is usually a homogeneously shaped Tensor of shape $[N, C, H, W, D]$ where $N$ would be the number of "grids" in our batch, the $C$ *channels* are equivalent to the size of the $E$ elements, and $H, W, D$ are the 3 index dimensions of a **dense** grid.

We can also directly use a `GridBatch` to derive a `JaggedTensor` to match the `GridBatch`'s specific sizing.

This more concise line of code has the same effect as the code above:

In [7]:
# From a GridBatch, a matching JaggedTensor can be created from a Tensor of shape `(total_voxels, feature_dim)`
features = grid_batch.jagged_like(torch.randn(grid_batch.total_voxels, 5, device=grid_batch.device))

Now that we have a `GridBatch` and a `JaggedTensor` of elements that correspond to the batch of grids, we could use the $ijk$ offsets we can obtain from the `GridBatch` to index into the `JaggedTensor` to retrieve the data for the voxels at those $ijk$'s.

In [8]:
features[feature_indices].jdata

tensor([[ 0.2314,  0.3543, -0.4464, -1.3043, -0.2004],
        [ 1.2822,  0.5833,  0.5202,  0.2874, -1.0102],
        [-0.6075,  0.2705,  2.6664,  0.6931,  0.0638],
        [ 1.4288,  0.7259,  0.6825, -0.3944, -0.6059],
        [ 1.9628,  0.5206,  0.9744,  0.9233, -1.0599],
        [ 0.1671,  0.1588,  1.0891, -1.0431,  0.7525],
        [ 0.6019, -0.2995,  1.8935,  0.4532, -0.1608],
        [ 1.0094,  0.5069,  0.2051,  0.1202, -0.7838]], device='cuda:0')

But beyond just looking up the elements directly, we can use higher-level ƒVDB functionality like sampling what the values are at a given worldspace position.

Let's get the data values at the worldspace positions used earlier in the lesson:

In [14]:
sampled_features = grid_batch.sample_trilinear(world_positions, features)


for xyz, value in zip(world_positions.jdata, sampled_features.jdata):
    print(f"World position {[round(num, 1) for num in xyz.tolist()]} \n\thas the values: {[round(num, 3) for num in value.tolist()]}")

World position [315.8, -125.8, -14.9] 
	has the values: [2.924, -0.969, 1.498, 0.891, -0.507, 0.987, -0.504, -0.267, 0.044, 0.014, 1.327, 0.303, 0.396, 0.014, -0.919, -0.056, 0.187, 1.027, 1.654, -0.106, 1.279, 1.695, -0.335, -0.024, 0.477, -0.93, -0.288, -0.021, -1.86, -0.529, -0.652, 0.965, -0.539, -0.813, -0.191, 0.74, -0.097, -1.428, 1.463, -0.552, -0.314, -0.601, 1.253, -1.332, 0.548, -0.198, -0.258, 0.713, -1.523, 0.941, 1.271, -0.556, -1.744, -1.993, 2.215, 0.047, -1.439, 0.849, -1.311, -0.1, 0.507, 1.357, 1.809, -0.158, 0.59, 1.116, 0.32, 1.116, -1.458, 0.97, -0.113, -0.274, 0.806, -0.237, 2.3, 0.131, -0.394, -0.473, 0.899, -1.126, 1.059, 0.777, 0.878, 0.695, 1.818, -0.334, -2.8, 1.102, -1.446, 2.523, -0.578, -0.766, -1.062, 0.038, 0.806, -0.832, 0.536, -0.574, 0.153, 1.091, -0.16, -2.215, 0.054, -1.426, 0.472, -0.082, 1.681, 0.482, -1.556, -0.209, -1.056, -0.581, -1.32, -1.058, 0.364, 0.339, -1.477, -0.189, -1.292, 0.174, 0.647, 0.54, 0.178, -0.855, 0.739, 0.866, -1.147, -0.07

### JaggedTensor Implementation Details


Internally, `JaggedTensor` does not represent the list of features as a list of differently sized PyTorch tensors, but instead stores a single tensor of the element data accompanied by a special index structure that allows us to access the data for each grid in the `GridBatch`.  

The `jdata` attribute of a `JaggedTensor` contains all the element values in this single list.  `jdata` is a `Tensor` of shape $[N, E]$ where $N$ is the total number of active voxels in the batch and $E$ is the element size.  

`jdata`'s shape would be equivalent to the result of concatenating the list of heterogeneously sized Tensors, mentioned above, along their first axis into a single Tensor whose shape would be $[B1+B2+B3…+Bn, E]$.

<center>

<img src = "img/jdata.jpg" width=1200 alt="jdata">

</center>

In [15]:
print(f"size of element data for the entire GridBatch: {features.jdata.shape}")

size of element data for the entire GridBatch: torch.Size([4133, 192])


To determine which grid each element belongs to, `JaggedTensor` contains indexing information in its `jidx` attribute.

`jidx` is a `Tensor` of shape $[N]$ where $N$ is again the total number of active voxels across the batch.  Each member of `jidx` is an integer that tells us which grid in the `GridBatch` the corresponding element in `jdata` belongs to.  The grid membership in `jidx` is ordered starting from 0 and members of the same batch are contiguous.

<center>

<img src = "img/jidx.jpg" width=1200 alt="jdata">

</center>

In [17]:
print(f"per-element membership for each voxel in the GridBatch: {features.jidx}")
print(f"\nthe size of the elements of the 4th grid in this batch: {features.jdata[features.jidx==3].shape}")

per-element membership for each voxel in the GridBatch: tensor([0, 0, 0,  ..., 7, 7, 7], device='cuda:0', dtype=torch.int32)

the size of the elements of the 4th grid in this batch: torch.Size([978, 192])


Additionally, `JaggedTensor` has a `joffsets` attribute that can also be used to index into `jdata` to get the element data for each grid in the batch.

The `joffsets` attribute is a `Tensor` of shape $[B]$, where $B$ is the number of grids in the batch.  `joffset`'s values are the start offsets into `jdata` that corresponds to each grid in the batch.  This is essentially the same information that can be found in `jidx` but expressed in a different form.

<center>

<img src = "img/joffsets.jpg" width=1200 alt="jdata">

</center>

In [18]:
print("per-grid offsets into the data:")
print(features.joffsets)
print(f"\nthe size of the elements of the 4th grid in this batch: {features.jdata[features.joffsets[3]:features.joffsets[3+1]].shape}")

per-grid offsets into the data:
tensor([   0,  154,  639,  984, 1962, 2499, 2790, 3784, 4133], device='cuda:0')

the size of the elements of the 4th grid in this batch: torch.Size([978, 192])


The element data stored in a `JaggedTensor` can be of any type that PyTorch supports, including float, float64, float16, bfloat16, int, etc., and the elements can have any arbitrary size.  In fact, elements can contain multi-dimensional tensor data.

For instance, there could be a `JaggedTensor` with 1 float that represents a signed distance field in each grid, or float elements of size 3 that represent an RGB color in each voxel of the grids, or elements of shape (3,3) representing 3x3 matrices, or float elements of length 192 that represent a learned feature vector of each voxel in each grid.

In [20]:
# A single scalar element per voxel
features = grid_batch.jagged_like(torch.randn(grid_batch.total_voxels, 1, dtype=torch.float, device=grid_batch.device))

# Cast to a double
features = features.double()

# A JaggedTensor of 3x3 matrices for element data
features = grid_batch.jagged_like(torch.randn(grid_batch.total_voxels, 3, 3, dtype=torch.float, device=grid_batch.device))

# A JaggedTensor of 192 float elements
features = grid_batch.jagged_like(torch.randn(grid_batch.total_voxels, 192, dtype=torch.float, device=grid_batch.device))